In [1]:
%pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]
%pip install gym[accept-rom-license]
%pip install pyglet


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gym
import random

C:\Users\BNPC\anaconda3\envs\PAcman\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [3]:
env = gym.make('MsPacman-v0')#, render_mode='human')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [4]:
env.unwrapped.get_action_meanings()

['NOOP',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'DOWNRIGHT',
 'DOWNLEFT']

In [5]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice(range(actions))
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:220.0
Episode:2 Score:280.0
Episode:3 Score:210.0
Episode:4 Score:220.0
Episode:5 Score:250.0


In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam, SGD

In [7]:
def build_model(height, width, channels, actions):
    model = Sequential(layers=[
        Convolution2D(32, (8, 8), strides = (4,4), activation='relu', input_shape = (3,height,width,channels)),
        Convolution2D(64, (4, 4), strides = (2,2), activation='relu'),
        Convolution2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(actions, activation='linear')])
    print(model.summary())
    return model


In [19]:
del model

In [20]:
model = build_model(height, width, channels, actions)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_10 (Dense)             (None, 9)                

In [21]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [22]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(
        model=model, 
        memory=memory, 
        policy=policy, 
        enable_dueling_network=True, 
        dueling_type='avg', 
        nb_actions=actions,
        nb_steps_warmup=1000
    )
    return dqn

In [23]:
dqn = build_agent(model, actions)

In [24]:
dqn.compile(Adam(learning_rate=1e-4))

In [25]:
dqn.fit(env, nb_steps=10000, visualize=True, verbose=2)

Training for 10000 steps ...
  818/10000: episode: 1, duration: 19.205s, episode steps: 818, steps per second:  43, episode reward: 260.000, mean reward:  0.318 [ 0.000, 10.000], mean action: 3.971 [0.000, 8.000],  loss: --, mean_q: --, mean_eps: --
done, took 141.580 seconds
